## 1、 什么是递归神经网络

为什么有BP神经网络、CNN，还需要RNN?

我们知道BP参数量大，通过CNN权值共享和局部连接可以减少参数，使得网络可以加深，因此有了CNN，那为什么会有RNN呢。
- BP神经网络和CNN的输入输出都是互相独立的；但是实际应用中有些场景输出内容和之前的内容是有关联的。
- RNN引入“记忆”的概念；递归指其每一个元素都执行相同的任务，但是输出依赖于输入和‘记忆’

BP神经网络和卷积神经网络有一个特点就是假设输入是一个独立的没有上下文联系的单位，比如输入时一张
图片，网络识别是狗还是猫，但是对于一些有明显的上下文特征的序列化输入，比如预测视屏中下一帧的播放内容，
那么很明显这样的输出必须依赖以前的输入，也就是说网络必须拥有一定的“记忆能力”，为了赋予网络这样的记忆力，一种
特殊结果的神经网络- 递归神经网络（Recurrent Neural Network）便应运而生。

递归神经网络的应用场景：
- 自然语言处理（NLP)
    - 语言模型与文本生成
- 机器翻译
- 语言识别
- 图像描述生成
- 文本相识度计算等

### 2、循环神经网络RNN-结构

![RNN-01.png](./pics/RNN-01.png)

上图中左边是RNN模型没有按时间展开的图，如果按时间序列展开，则是上图中的右边部分，我们重点观察右边部分的图。
这幅图描述了序列索引号$t$附近RNN的模型，其中：

- $x^{(t)}$代表在序列索引号$t$时训练样本的输入，同样的 $x^{(t-1)}$和$x^{(t+1)}$代表在序列索引号$t-1$和$t+1$时训练样本的输入。

- $h^{(t)}$代表序列索引号$t$时模型的隐藏状态。$h^{(t)}$由$x^{(t)}$和$h^{(t-1)}$共同决定。

- $o^{(t)}$代表在序列索引号$t$时模型的输出，$o^{(t)}$只由模型当前的隐藏状态$h^{(t)}$决定。

- $L^{(t)}$代表在序列索引号$t$时模型的损失函数。

- $y^{(t)}$代表在序列号$t$时训练样本序列的真实输出。

- $U,W,V$这三个矩阵使我们的模型的线形关系参数，它在整个RNN网络中是共享的，这点和DNN很不相同，也正是因为是共享了，它体现了RNN
的模型的"循环反馈"的思想。

## 3、RNN前向传播算法

有了上面的模型，RNN的前向传播的算法就很容易得到
对于任意一个序列索引号$t$，我们隐藏状态$h^{(t)}$由$x^{(t)}$和$h^{(t-1)}$得到：

$$
h^{(t)} = \sigma (z^{(t)})) = \sigma\left(Ux^{(t)}+W(h^{(t-1)}) + b) \right) = \tanh \left(Ux^{(t)}+W(h^{(t-1)}) + b) \right)  ……式(1.1)
$$

其中$\sigma$为RNN的激活函数，一般为tanh, b为偏置值。
其中$(\tanh(z))' = 1-(h(z))^2$……式(1.2)

 
$$
\begin{aligned}
\frac{\partial{h^{(t)}}}{\partial{W}}=&\tanh(h^{(t)}))'h^{(t-1)}\\
=& (1-(h^{(t)})^2)h^{(t-1)}
\end{aligned}
$$
 

序列索引号$t$时模型的输出$o^{(t)})$的表达式为：

$$
o^{(t)} = Vh^{(t)} + c  ……式(1.3)
$$

最终在序列索引号$t$我们的预测输出为：
$$
\hat {y} ^{(t)}  = softmax\left(o^{(t)}\right)   ……式(1.4)
$$

Softmax 将连续数值转化成相对概率

通过算是函数$L^{(t)}$,比如对数似然损失函数，我们可以量化模型在当前位置的损失即$\hat {y}^{(t)}$和$y^{(t)}$的差距

## 4、RNN反向传播算法

有了RNN前向传播算法的基础，就可以推导RNN反向传播算法的流程了。RNN反向传播算法的思路是一样的。即通过梯度下降法一轮轮的迭代，得到合适的RNN
模型参数$U,W,V,b,c$。由于RNN是基于时间反向传播，所以RNN的反向传播有时也叫做BPTT(back-propagation through time)
但是BPTT和BP也有很大的不同点，即这里所有的$U,W,V,b,c$在序列的各个位置是共享的，反向传播时我们更新的是相同的参数。

为了简化描述，这里的损失函数我们为对数损失函数，输出的激活函数为softmax函数，隐藏层的激活函数为tanh函数。

对于RNN，由于在序列的每个位置都有损失函数，因此最终的损失$L$为：
$$
Lost = \sum_{t=1}^{\tau}(L^{(t)})
$$

在计算梯度之前，有必要先了解一下激活函数softmax结合对数损失的求导过程.[传送门](./03-softmax的对数似然损失函数求导.ipynb)

其中$V,c$的梯度计算是比较简单的：

$$
\begin{aligned}
\frac{\partial{Lost}}{\partial{c}} =& \sum_{t=1}^{\tau}\frac{\partial{L^{(t)}}}{\partial{c}}\\
=&\sum_{t=1}^{\tau}\frac{\partial{L^{(t)}}}{\partial{o}}\frac{\partial{o^{(t)}}}{\partial{c}}\\
=&\sum_{t=1}^{\tau}\hat {y}^{(t)} - y^{(t)}  ……式(1.5)
\end{aligned}
$$

$$
\begin{aligned}
\frac{\partial{Lost}}{\partial{V}} =& \sum_{t=1}^{\tau}\frac{\partial{L^{(t)}}}{\partial{V}}\\
=&\sum_{t=1}^{\tau}\frac{\partial{L^{(t)}}}{\partial{o}}\frac{\partial{o^{(t)}}}{\partial{c}}\\
=&\sum_{t=1}^{\tau}\left(\hat {y}^{(t)} - y^{(t)}\right)\left(h^{(t)}\right)^{T} ……式(1.6)
\end{aligned}
$$


接下来计算$W，U,b$的梯度，从RNN的模型可以看出，在反向传播时，在某一序列位置$t$的梯度损失由当前位置的输出对应的梯度损失
和$t+1$时的梯度损失两部分共同决定，对于$W$在某一序列位置$t$的梯度损失需要反向传播一步步的计算，我们定义序列索引$t$位置的隐藏状态的梯度为：

$$
\delta^{(t)} = \frac{\partial{Lost}}{\partial{h^{(t)}}}
$$

这样我们可以像BP一样从$\delta^{(t+1)}$递推$\delta^{(t)}$：

$$
\begin{aligned}
\delta^{(t)} =& \frac{\partial{Lost}}{\partial{h^{(t)}}}\\
=&\frac{\partial{Lost}}{\partial{o^{(t)}}}\frac{\partial{o^{(t)}}}{\partial{h^{t}}}+\frac{\partial{Lost}}{h^{(t+1)}}\frac{\partial{h^{t+1}}}{\partial{h^{(t)}}}\\
=&V^{T}(\hat {y}^{(t)}-y^{(t)}) + W^{T}\delta^{(t+1)}diag(1-(h^{(t+1)})^2)
\end{aligned}
$$

对于$\delta^{(t)}$,由于它的后面没有其他的序列索引，因此有：
$$
\begin{aligned}
\delta^{(\tau)} =& \frac{\partial{Lost}}{\partial{o^{(\tau)}}}\frac{\partial{o^{(\tau)}}}{\partial{h^{(\tau)}}}\\
=&V^{T}(\hat {y}^{(t)} - y^{(\tau)})
\end{aligned}
$$

有了$\delta^{(t)}$ ，计算$W,U,b$就容易了，这里给出$W,U,b$的梯度计算表达式：

$$
\begin{aligned}
\frac{\partial{Lost}}{\partial{W}} =&\sum_{t=1}{\tau}\frac{\partial{L}}{\partial{h^{(t)}}}\frac{\partial{h^{(t)}}}{\partial{W}}\\
=&\sum_{t=1}^{\tau}\frac{\partial{L}}{\partial{o^{(t)}}}\frac{\partial{o^{(t)}}}{\partial{h^{(t)}}}\frac{\partial{h^{(t)}}}{\partial{W}}\\
=&\sum_{t=1}^{\tau}diag(1-(h^{(t)})^2){\delta}^{(t)}(h^{(t-1)})^T
\end{aligned}
$$

$$
\begin{aligned}
\frac{\partial{Lost}}{\partial{b}} =&\sum_{t=1}{\tau}\frac{\partial{L}}{\partial{h^{(t)}}}\frac{\partial{h^{(t)}}}{\partial{b}}\\
=&\sum_{t=1}^{\tau}\frac{\partial{L}}{\partial{o^{(t)}}}\frac{\partial{o^{(t)}}}{\partial{h^{(t)}}}\frac{\partial{h^{(t)}}}{\partial{b}}\\
=&\sum_{t=1}^{\tau}diag(1-(h^{(t)})^2){\delta}^{(t)}
\end{aligned}
$$

$$
\begin{aligned}
\frac{\partial{Lost}}{\partial{U}} =&\sum_{t=1}{\tau}\frac{\partial{L}}{\partial{h^{(t)}}}\frac{\partial{h^{(t)}}}{\partial{U}}\\
=&\sum_{t=1}^{\tau}\frac{\partial{L}}{\partial{o^{(t)}}}\frac{\partial{o^{(t)}}}{\partial{h^{(t)}}}\frac{\partial{h^{(t)}}}{\partial{U}}\\
=&\sum_{t=1}^{\tau}diag(1-(h^{(t)})^2){\delta}^{(t)}(x^{(t)})^T
\end{aligned}
$$

## RNN总结

RNN理论上解决序列数据的训练，但是它也像DNN一样有梯度消失的问题，当序列很长的时候问题尤其严重
因此，上面的RNN模型一般不能直接用于应用领域，在语音识别手写识别以及机器翻译等NLP领域实际上应用比较
广泛的是基于RNN模型的一个特例LSTM .[传送门](./02-LSTM模型前向反向传播算法.ipynb)